Opioid Settlement Payouts Data from KFF Health News https://kffhealthnews.org/opioid-settlements/

In [37]:
import pandas as pd
import numpy as np

import os

In [38]:
# Set pandas display option to format floats with commas
pd.options.display.float_format = '{:,.2f}'.format

In [39]:
# Load the Excel file
file_path = 'Opioid-Settlement-Payouts-Data_040124.xlsx'
xls = pd.ExcelFile(file_path)  #creates an object to hold the information

# Define the sheet(s) to exclude
sheets_to_exclude = ['Terms of Use']  

# List to hold individual DataFrames
dfs = []

# Loop through the sheets
for sheet_name in xls.sheet_names:
    if sheet_name not in sheets_to_exclude:
        # Read each sheet into a DataFrame
        df = pd.read_excel(xls, sheet_name=sheet_name)
        
        # add a column to identify the sheet
        df['Sheet'] = sheet_name
        
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dfs, ignore_index=True)

# Display the first few rows of the combined DataFrame
print(combined_df.head())

         name  2022 settlement payout received  \
0     Alabama                              NaN   
1      Alaska                    12,659,625.14   
2     Arizona                    64,711,719.27   
3    Arkansas                    46,532,891.66   
4  California                   282,165,225.08   

   2023 settlement payout received  2024 estimated future payout  \
0                              NaN                           NaN   
1                     2,152,924.07                  2,780,784.41   
2                    32,636,542.16                 44,724,172.02   
3                     8,049,659.95                 10,075,289.71   
4                   137,461,655.35                168,369,723.68   

   2025 estimated future payout  2026 estimated future payout  \
0                           NaN                           NaN   
1                  2,694,689.45                  3,113,944.04   
2                 46,071,147.49                 28,570,326.56   
3                 10,075,289.7

Alabama and West Virginia data contains N/A.  Drop these states.

In [40]:
filtered_df = combined_df[~combined_df['name'].isin(['Alabama', 'West Virginia'])]
print(filtered_df.head())

         name  2022 settlement payout received  \
1      Alaska                    12,659,625.14   
2     Arizona                    64,711,719.27   
3    Arkansas                    46,532,891.66   
4  California                   282,165,225.08   
5    Colorado                    44,778,002.07   

   2023 settlement payout received  2024 estimated future payout  \
1                     2,152,924.07                  2,780,784.41   
2                    32,636,542.16                 44,724,172.02   
3                     8,049,659.95                 10,075,289.71   
4                   137,461,655.35                168,369,723.68   
5                    14,531,328.42                 31,315,104.88   

   2025 estimated future payout  2026 estimated future payout  \
1                  2,694,689.45                  3,113,944.04   
2                 46,071,147.49                 28,570,326.56   
3                 10,075,289.71                 11,629,948.67   
4                173,279,465.9